In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix
from contemporary_ner_training.conll_ner_importer import conll_to_ner_labelling  
from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

CONLL NER test:

In [3]:
'''
texts = []
for file in os.listdir('contemporary_ner_training'):
    if file.startswith('estner_split_1'):
        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               
'''

"\ntexts = []\nfor file in os.listdir('contemporary_ner_training'):\n    if file.startswith('estner_split_1'):\n        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               \n"

Flags:

In [4]:
use_vabamorfcorpustagger = False

---

In [5]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [6]:
def create_training_texts(filenames):
    print("Valmistan ette treenimistekste")
    
    # These files don't work because the protocols are written in a different language,
    # which the goldstandard didn't recognise, hence have no goldstandard tags.
    files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']
    
    start = time.time()
    training_texts = []
    for filename in filenames:
        with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
            if filename in files_not_working:
                continue
            else:
                tagged_text = preprocess_text(json_to_text(file.read()))
                if use_vabamorfcorpustagger:
                    tagged_text.pop_layer('morph_analysis')
                    vm_corpus_tagger.tag([tagged_text])
                training_texts.append(tagged_text)
    print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
    return training_texts

In [7]:
def train_nertagger(training_texts, new_model_dir):
    print("Alustan NerTaggeri treenimist.")
    start = time.time()
    
    modelUtil = ModelStorageUtil( new_model_dir )
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
    print(f"NerTagger treenitud {time.time() - start} sekundiga.")

In [8]:
def get_testing_and_training_subdistribution(subdistribution):
    training = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            testing = y
        else:
            training.append(y)
    return testing, training

In [9]:
def tag_files(model_dir, testing_files, nertagger, use_vabamorfcorpustagger):
    removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']
    
    print("\n\nAlustan failide märgendamist.")
    start = time.time()
    
    for test_file in testing_files:
        with open(find(test_file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
            text = f.read()
        
        if test_file == "Tartu_V6nnu_Ahja_id3502_1882a.json":
            text = text.replace('..', '. .')
        text = preprocess_text(Text(text))

        if use_vabamorfcorpustagger:
            text.pop_layer('morph_analysis')
            text = [text]
            vm_corpus_tagger.tag( text )
            text = text[0]
        nertagger.tag(text)
        text.add_layer(flatten(text['ner'], 'flat_ner'))

        for x in removed_layers:
            text.pop_layer(x)

        path = os.path.join(model_dir, 'vallakohtufailid-trained-nertagger')
        if not os.path.exists(path):
            os.mkdir(path)
            
        text_to_json(text, file=os.path.join(model_dir, 'vallakohtufailid-trained-nertagger', test_file))

            
        print(f'Märgendatud fail {test_file}')
    print(f"Failid märgendatud {time.time() - start} sekundiga.")

In [10]:
def train_model(model_directory):
    for subdistribution in [1, 2, 3, 4, 5]:
        testing, training = get_testing_and_training_subdistribution(subdistribution)
        
        # Get the filenames to be trained on from the files dictionary
        filenames = [key for key, value in files.items() if int(value) in training]

        # Create training_texts from the aforementioned filenames
        training_texts = create_training_texts(filenames)
        
        # Set up the trainer and training
        new_model_dir = os.path.join('models', model_directory)
        train_nertagger(training_texts, new_model_dir)
        
        # Set up the new trained nertagger and defining layers to be removed later on
        tagger = NerTagger(model_dir = new_model_dir)
        
        # Tag the files using the new nertagger
        testing_files = [key for key, value in files.items() if int(value) == testing]
        tag_files(new_model_dir, testing_files, tagger, use_vabamorfcorpustagger)
            
    # Get results of model
    extract_results_to_txt_file(model_directory, files)
    
    print(f"Mudel {model_directory} treenitud.")

To train the model the `model_directory` must contain a `settings.py` file

In [12]:
train_model(os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features','model_gazetteer_both_new'))

Valmistan ette treenimistekste
Treenimistekstid ette valmistatud 103.38949155807495 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 347398
Seconds required: 1.274

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 4716.521098
Trial #2 (eta = 0.200000): 7963.921876
Trial #3 (eta = 0.400000): 14993.010023
Trial #4 (eta = 0.800000): 28508.454329
Trial #5 (eta = 1.600000): 45844.605022 (worse)
Trial #6 (eta = 0.050000): 3520.310148
Tria

***** Epoch #39 *****
Loss: 859.714826
Improvement ratio: 0.353073
Feature L2-norm: 90.092984
Learning rate (eta): 0.024951
Total number of feature updates: 633477
Seconds required for this iteration: 0.459

***** Epoch #40 *****
Loss: 833.842091
Improvement ratio: 0.346102
Feature L2-norm: 90.728224
Learning rate (eta): 0.024950
Total number of feature updates: 649720
Seconds required for this iteration: 0.457

***** Epoch #41 *****
Loss: 821.161730
Improvement ratio: 0.319791
Feature L2-norm: 91.349373
Learning rate (eta): 0.024949
Total number of feature updates: 665963
Seconds required for this iteration: 0.459

***** Epoch #42 *****
Loss: 801.995280
Improvement ratio: 0.307888
Feature L2-norm: 91.953071
Learning rate (eta): 0.024948
Total number of feature updates: 682206
Seconds required for this iteration: 0.466

***** Epoch #43 *****
Loss: 783.237582
Improvement ratio: 0.292873
Feature L2-norm: 92.545692
Learning rate (eta): 0.024946
Total number of feature updates: 698449
Seco

***** Epoch #79 *****
Loss: 439.614803
Improvement ratio: 0.132970
Feature L2-norm: 107.867610
Learning rate (eta): 0.024902
Total number of feature updates: 1283197
Seconds required for this iteration: 0.465

***** Epoch #80 *****
Loss: 435.967560
Improvement ratio: 0.124870
Feature L2-norm: 108.186565
Learning rate (eta): 0.024900
Total number of feature updates: 1299440
Seconds required for this iteration: 0.460

***** Epoch #81 *****
Loss: 429.547547
Improvement ratio: 0.136596
Feature L2-norm: 108.500771
Learning rate (eta): 0.024899
Total number of feature updates: 1315683
Seconds required for this iteration: 0.486

***** Epoch #82 *****
Loss: 430.701944
Improvement ratio: 0.105934
Feature L2-norm: 108.811192
Learning rate (eta): 0.024898
Total number of feature updates: 1331926
Seconds required for this iteration: 0.604

***** Epoch #83 *****
Loss: 419.802564
Improvement ratio: 0.144872
Feature L2-norm: 109.115654
Learning rate (eta): 0.024897
Total number of feature updates: 13

***** Epoch #118 *****
Loss: 317.807807
Improvement ratio: 0.062578
Feature L2-norm: 117.984569
Learning rate (eta): 0.024853
Total number of feature updates: 1916674
Seconds required for this iteration: 0.466

***** Epoch #119 *****
Loss: 319.607057
Improvement ratio: 0.048914
Feature L2-norm: 118.197710
Learning rate (eta): 0.024852
Total number of feature updates: 1932917
Seconds required for this iteration: 0.457

***** Epoch #120 *****
Loss: 311.108665
Improvement ratio: 0.077410
Feature L2-norm: 118.408446
Learning rate (eta): 0.024851
Total number of feature updates: 1949160
Seconds required for this iteration: 0.459

***** Epoch #121 *****
Loss: 310.941548
Improvement ratio: 0.058787
Feature L2-norm: 118.615924
Learning rate (eta): 0.024850
Total number of feature updates: 1965403
Seconds required for this iteration: 0.456

***** Epoch #122 *****
Loss: 309.072144
Improvement ratio: 0.076467
Feature L2-norm: 118.821965
Learning rate (eta): 0.024848
Total number of feature update

***** Epoch #161 *****
Loss: 254.716761
Improvement ratio: 0.014883
Feature L2-norm: 125.788219
Learning rate (eta): 0.024800
Total number of feature updates: 2615123
Seconds required for this iteration: 0.462

***** Epoch #162 *****
Loss: 247.278871
Improvement ratio: 0.057667
Feature L2-norm: 125.943913
Learning rate (eta): 0.024799
Total number of feature updates: 2631366
Seconds required for this iteration: 0.458

***** Epoch #163 *****
Loss: 246.697721
Improvement ratio: 0.050148
Feature L2-norm: 126.099089
Learning rate (eta): 0.024798
Total number of feature updates: 2647609
Seconds required for this iteration: 0.466

***** Epoch #164 *****
Loss: 248.791159
Improvement ratio: 0.039745
Feature L2-norm: 126.252147
Learning rate (eta): 0.024797
Total number of feature updates: 2663852
Seconds required for this iteration: 0.457

***** Epoch #165 *****
Loss: 245.908198
Improvement ratio: 0.046175
Feature L2-norm: 126.405010
Learning rate (eta): 0.024795
Total number of feature update

...

Märgendatud fail Tartu_Kodavere_Pala_id23275_1872a.json
Märgendatud fail L22ne_Reigi_K6rgessaare_id23087_1894a.json
Märgendatud fail Tartu_N6o_Aru_id5374_1890a.json
Märgendatud fail Harju_Juuru_Juuru_id20228_1868a.json
Märgendatud fail Tartu_V6nnu_Rasina_id13313_1865a.json
Märgendatud fail Tartu_V6nnu_Ahja_id15819_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13002_1875a.json
Märgendatud fail Tartu_Kodavere_Pala_id21493_1869a.json
Märgendatud fail V6ru_Vastseliina_Misso_id19537_1882a.json
Märgendatud fail Saare_Mustjala_Mustjala_id7076_1821a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24033_1888a.json
Märgendatud fail Viru_Rakvere_S6meru_id10293_1883a.json
Märgendatud fail Tartu_Torma_Avinurme_id24697_1824a.json
Märgendatud fail Harju_Kose_Palvere_id13988_1880a.json
Märgendatud fail Tartu_V6nnu_Ahja_id12425_1875a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id4649_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20421_1888a.json
Märgendatud fail Harju_Kose_Palvere_id19467_1884a.json
M

***** Epoch #28 *****
Loss: 608.027159
Improvement ratio: 0.573759
Feature L2-norm: 101.436786
Learning rate (eta): 0.049860
Total number of feature updates: 437192
Seconds required for this iteration: 0.480

***** Epoch #29 *****
Loss: 592.960535
Improvement ratio: 0.531213
Feature L2-norm: 102.294547
Learning rate (eta): 0.049855
Total number of feature updates: 452806
Seconds required for this iteration: 0.484

***** Epoch #30 *****
Loss: 569.000076
Improvement ratio: 0.485823
Feature L2-norm: 103.121077
Learning rate (eta): 0.049850
Total number of feature updates: 468420
Seconds required for this iteration: 0.485

***** Epoch #31 *****
Loss: 568.356369
Improvement ratio: 0.443892
Feature L2-norm: 103.917878
Learning rate (eta): 0.049845
Total number of feature updates: 484034
Seconds required for this iteration: 0.480

***** Epoch #32 *****
Loss: 541.693469
Improvement ratio: 0.442090
Feature L2-norm: 104.692505
Learning rate (eta): 0.049841
Total number of feature updates: 499648

***** Epoch #70 *****
Loss: 295.522039
Improvement ratio: 0.070194
Feature L2-norm: 123.823570
Learning rate (eta): 0.049652
Total number of feature updates: 1092980
Seconds required for this iteration: 0.481

***** Epoch #71 *****
Loss: 291.536791
Improvement ratio: 0.109977
Feature L2-norm: 124.166470
Learning rate (eta): 0.049648
Total number of feature updates: 1108594
Seconds required for this iteration: 0.479

***** Epoch #72 *****
Loss: 276.849031
Improvement ratio: 0.148090
Feature L2-norm: 124.500672
Learning rate (eta): 0.049643
Total number of feature updates: 1124208
Seconds required for this iteration: 0.486

***** Epoch #73 *****
Loss: 275.031588
Improvement ratio: 0.147148
Feature L2-norm: 124.835812
Learning rate (eta): 0.049638
Total number of feature updates: 1139822
Seconds required for this iteration: 0.484

***** Epoch #74 *****
Loss: 279.923684
Improvement ratio: 0.064256
Feature L2-norm: 125.172536
Learning rate (eta): 0.049633
Total number of feature updates: 11

Märgendatud fail V6ru_Vastseliina_Misso_id8101_1885a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13681_1886a.json
Märgendatud fail Tartu_N6o_Pangodi_id5262_1889a.json
Märgendatud fail Harju_Keila_Saue_id13436_1885a.json
Märgendatud fail Harju_Kose_Triigi_id9536_1868a.json
Märgendatud fail Tartu_Torma_Avinurme_id21275_1871a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id12075_1885a.json
Märgendatud fail L22ne_Martna_Martna_id23117_1873a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id6881_1877a.json
Märgendatud fail Viru_Haljala_Vihula_id4974_1876a.json
Märgendatud fail P2rnu_Tori_Tori_id25322_1889a.json
Märgendatud fail Tartu_Kodavere_Ranna_id19673_1865a.json
Märgendatud fail J2rva_Tyri_S2revere_id9244_1885a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id22818_1882a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16349_1884a.json
Märgendatud fail Harju_Kuusalu_Kolga_id11902_1888a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22150_1879a.json
Märgendatud fail Tartu_Kodavere_Pala_i

Märgendatud fail J2rva_Tyri_V22tsa_id16931_1886a.json
Märgendatud fail Harju_Harju-Madise_Padise-Kloostri_id2778_1867a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id4977_1883a.json
Märgendatud fail J2rva_Tyri_S2revere_id8673_1886a.json
Märgendatud fail V6ru_R2pina_Kahkva_id14118_1889a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id22309_1867a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id25096_1877a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id7206_1876a.json
Märgendatud fail V6ru_R2pina_Kahkva_id6348_1887a.json
Märgendatud fail Tartu_N6o_Meeri_id4980_1895a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14731_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id23690_1866a.json
Märgendatud fail Harju_Kose_Palvere_id17074_1882a.json
Märgendatud fail Viljandi_Paistu_Holstre_id515_1843a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id11749_1885a.json
Märgendatud fail Harju_Harju-Madise_Padise-Kloostri_id3283_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id15058_1883a.json
Märgendatud fail